<a href="https://colab.research.google.com/github/aashinsravan/Deep-Learning/blob/main/Project_1_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Installation

In [ ]:
 ! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Mounting the Drive and Loading the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir ~/.kaggle


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp /content/drive/MyDrive/Colab\ Notebooks/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Downloading the dataset
!kaggle datasets download -d abdelghaniaaba/wildfire-prediction-dataset

wildfire-prediction-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#Unzipping the dataset
!unzip wildfire-prediction-dataset.zip -d /content

Archive:  wildfire-prediction-dataset.zip
replace /content/test/nowildfire/-113.91777,50.901087.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Loading Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers import SGD, adam


# Pre-Processing

Parameters

In [ ]:
EPOCH= 10
BATCH_SIZE= 64
TRGT_SIZE =(64,64)

Assigning the paths

In [ ]:
train_path='/content/train'
test_path='/content/test'
val_path='/content/valid'

Setting up image data generator

In [ ]:
train_datagen= ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2, 
                                  zoom_range=0.2, 
                                  horizontal_flip=True,
                                  rotation_range=75)

test_datagen= ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(train_path, 
                                                  target_size=TRGT_SIZE, 
                                                  batch_size=BATCH_SIZE, 
                                                  class_mode='binary')

val_generator = test_datagen.flow_from_directory(val_path,
                                                  target_size=TRGT_SIZE,
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_path,
                                                   target_size=TRGT_SIZE,
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='binary')


# Defining the Model

In [ ]:
model=Sequential()

model.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu',  kernel_initializer='he_uniform'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu',  kernel_initializer='he_uniform'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()


# Compiling the Model


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile( optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Training the Model

In [ ]:
X_train, y_train = next(train_generator)
X_valid, y_valid = next(val_generator)
print(f"Train data shape: {X_train.shape}")
print(f"Train labels shape: {y_train.shape}")
print(f"Validation data shape: {X_valid.shape}")
print(f"Validation labels shape: {y_valid.shape}")

In [ ]:
checkpoint = ModelCheckpoint('model.hdf5',verbose=1, save_best_only= True)
early_stopping = EarlyStopping(monitor= 'val_loss', patience= 10)
with tf.device('/GPU:0'):
  history = model.fit( train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data = val_generator,
                    validation_steps=len(val_generator),
                    callbacks=[checkpoint, early_stopping])

# Plotting

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label = 'train',)
plt.plot(history.history['val_accuracy'], label = 'valid')

# adding legend and labels
plt.legend(loc = 'lower right')
plt.xlabel('epochs')
plt.ylabel('accuracy')

# show the plot
plt.show()   

# Testing the Test Set

Classification Report

In [ ]:
from sklearn.metrics import classification_report
# Load the saved model
model.load_weights('model.hdf5')

# Evaluate the model on the test set
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print(f"Test loss: {scores[0]}")
print(f"Test accuracy: {scores[1]}")

# Predict the labels of the test set
y_pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred = (y_pred > 0.5).astype(int)

# Compute evaluation metrics
print(classification_report(test_generator.classes, y_pred))


Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Load the saved model
model.load_weights('model.hdf5')

# Predict the labels of the test set
y_pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred = (y_pred > 0.5).astype(int)

# Compute the confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)

# Print the confusion matrix
print(cm)


In [ ]:
# Plotting confusion matrix
import seaborn as sns
cf_matrix = confusion_matrix(test_generator.classes, y_pred, normalize='true')
plt.figure(figsize = (17,12))
sns.heatmap(cf_matrix, annot=True, xticklabels = sorted(set(test_generator.classes)), yticklabels = sorted(set(test_generator.classes)),cbar=False)
plt.title('Normalized Confusion Matrix', fontsize = 23)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Actual", fontsize=15)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Load the saved model
model.load_weights('model.hdf5')

# Predict the labels of the test set
y_pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred = (y_pred > 0.5).astype(int)

# Compute the confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)

# Define class labels
class_names = ['Non-Fire', 'Fire']

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()
